# 시작

In [121]:
import os
import re
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds


# 데이터 준비 & 전처리

In [122]:
def load_data():
    path = os.getenv('HOME') + '/aiffel/songys_chatbot/ChatbotData.csv'
    data = pd.read_csv(path)
    print('데이터를 불러왔습니다: {}개 문장'.format(len(data)))
    print(data.head())
    return data


def preprocess_sentence(sentence):
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = sentence.strip()
    return sentence

In [123]:
MAX_LENGTH = 20

def tokenize_data(data):
    questions = data['Q'].apply(preprocess_sentence).tolist()
    answers = data['A'].apply(preprocess_sentence).tolist()
    
    tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(questions + answers, target_vocab_size=2**13)
    start_token, end_token = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]
    
    tokenized_questions = []
    tokenized_answers = []
    
    removed_count = 0
    for question, answer in zip(questions, answers):
        tokenized_question = start_token + tokenizer.encode(question) + end_token
        tokenized_answer = start_token + tokenizer.encode(answer) + end_token
        
        if len(tokenized_question) < MAX_LENGTH and len(tokenized_answer) < MAX_LENGTH:
            tokenized_questions.append(tokenized_question)
            tokenized_answers.append(tokenized_answer)
        else:
            removed_count += 1
    print('{}개 문장이 {}보다 길어 삭제되었습니다'.format(removed_count, MAX_LENGTH))
    tokenized_questions = tf.keras.preprocessing.sequence.pad_sequences(tokenized_questions, maxlen=MAX_LENGTH, padding='post')
    tokenized_answers = tf.keras.preprocessing.sequence.pad_sequences(tokenized_answers, maxlen=MAX_LENGTH, padding='post')
    return tokenizer, tokenized_questions, tokenized_answers

In [124]:
def make_dataset(questions, answers):
    BATCH_SIZE = 64
    BUFFER_SIZE = 20000
    return tf.data.Dataset.from_tensor_slices((
        {
            'inputs': questions,
            'dec_inputs': answers[:, :-1]
        },
        {
            'outputs': answers[:, 1:]
        },
    ))\
    .cache()\
    .shuffle(BUFFER_SIZE)\
    .batch(BATCH_SIZE)\
    .prefetch(tf.data.experimental.AUTOTUNE)

# 모델 생성 & 훈련 함수

In [125]:
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.pos_encoding = self.positional_encoding(vocab_size, d_model)
        
    def positional_encoding(self, vocab_size, d_model):
        angle_rads = self.get_angles(vocab_size, d_model)
        sins = tf.math.sin(angle_rads[:, 0::2])
        coss = tf.math.cos(angle_rads[:, 1::2])
        pos_encoding = tf.concat([sins, coss], axis=-1)[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)
        
    def get_angles(self, vocab_size, d_model):
        v_positions = tf.range(vocab_size, dtype=tf.float32)[:, tf.newaxis]
        d_positions = tf.range(d_model, dtype=tf.float32)[tf.newaxis, :]
        angles = 1 / tf.pow(10000, (2*(d_positions//2)) / tf.cast(d_model, tf.float32))
        return v_positions * angles
    
    def call(self, inputs):
        return inputs + self.pos_encoding[:, tf.shape(inputs)[1], :]

class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.pos_encoding = self.positional_encoding(vocab_size, d_model)

    def get_angles(self, positions, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return positions * angles

    def positional_encoding(self, max_position, d_model):
        angle_rads = self.get_angles(
            tf.range(max_position, dtype=tf.float32)[:, tf.newaxis],
            tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
            d_model)
        sines = tf.math.sin(angle_rads[:, 0::2])
        cosines = tf.math.cos(angle_rads[:, 1::2])

        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [126]:
data = load_data()

데이터를 불러왔습니다: 11823개 문장
                 Q            A  label
0           12시 땡!   하루가 또 가네요.      0
1      1지망 학교 떨어졌어    위로해 드립니다.      0
2     3박4일 놀러가고 싶다  여행은 언제나 좋죠.      0
3  3박4일 정도 놀러가고 싶다  여행은 언제나 좋죠.      0
4          PPL 심하네   눈살이 찌푸려지죠.      0


In [127]:
tokenizer, questions, answers = tokenize_data(data)

45개 문장이 20보다 길어 삭제되었습니다


In [128]:
VOCAB_SIZE = tokenizer.vocab_size + 2
print('단어장 크기는 {}입니다'.format(VOCAB_SIZE))

단어장 크기는 8175입니다


In [129]:
dataset = make_dataset(questions, answers)

In [130]:
def scaled_dot_product_attention(query, key, value, mask):
    matmul_qk = tf.matmul(query, key, transpose_b=True)

    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth)

    if mask is not None:
        logits += (mask * -1e9)

    attention_weights = tf.nn.softmax(logits, axis=-1)
    output = tf.matmul(attention_weights, value)
    return output


class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super().__init__(name=name)
        self.d_model = d_model
        self.num_heads = num_heads  
        assert d_model % self.num_heads == 0

        self.d_k = d_model // self.num_heads

        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)

        self.dense = tf.keras.layers.Dense(units=d_model)

    def split_heads(self, inputs, batch_size):
        inputs = tf.reshape(inputs, shape=(batch_size, -1, self.num_heads, self.d_k))
        return tf.transpose(inputs, perm=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs['value'], inputs['mask']
        batch_size = tf.shape(query)[0]

        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        scaled_attention = scaled_dot_product_attention(query, key, value, mask)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        concat_attention = tf.reshape(scaled_attention,(batch_size, -1, self.d_model))
        
        outputs = self.dense(concat_attention)
        return outputs

In [131]:
def create_padding_mask(x):
    return tf.cast(tf.math.equal(x, 0), tf.float32)[:, tf.newaxis, tf.newaxis, :]


def create_look_ahead_mask(x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    padding_mask = create_padding_mask(x)
    return tf.maximum(look_ahead_mask, padding_mask)


def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")
    attention_inputs = {
        'query': inputs,
        'key': inputs,
        'value': inputs,
        'mask': padding_mask
    }
    attention = MultiHeadAttention(d_model, num_heads, name="attention")(attention_inputs)

    attention = tf.keras.layers.Dropout(dropout)(attention)
    attention = tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs + attention)

    outputs = tf.keras.layers.Dense(units, activation='relu')(attention)
    outputs = tf.keras.layers.Dense(d_model)(outputs)

    outputs = tf.keras.layers.Dropout(dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention + outputs)

    return tf.keras.Model(inputs=[inputs, padding_mask], outputs=outputs, name=name)


def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="encoder"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = encoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name="encoder_layer_{}".format(i),
        )([outputs, padding_mask])

    return tf.keras.Model(inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [132]:
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
    look_ahead_mask = tf.keras.Input(shape=(1, None, None), name="look_ahead_mask")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    attention1_inputs = {
        'query': inputs,
        'key': inputs,
        'value': inputs,
        'mask': look_ahead_mask
    }
    attention1 = MultiHeadAttention(d_model, num_heads, name="attention_1")(inputs=attention1_inputs)
    attention1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention1 + inputs)

    attention2_inputs = {
        'query': attention1,
        'key': enc_outputs,
        'value': enc_outputs,
        'mask': padding_mask
    }
    attention2 = MultiHeadAttention(d_model, num_heads, name="attention_2")(inputs=attention2_inputs)
    attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
    attention2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention2 + attention1)

    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)

    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(outputs + attention2)

    return tf.keras.Model(inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask], outputs=outputs, name=name)


def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):
    inputs = tf.keras.Input(shape=(None,), name='inputs')
    enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
    look_ahead_mask = tf.keras.Input(shape=(1, None, None), name='look_ahead_mask')

    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')
  
    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = decoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name='decoder_layer_{}'.format(i),
        )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

    return tf.keras.Model(inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask], outputs=outputs, name=name)

In [133]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name="transformer"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

    enc_padding_mask = tf.keras.layers.Lambda(create_padding_mask, output_shape=(1, 1, None),name='enc_padding_mask')(inputs)
    look_ahead_mask = tf.keras.layers.Lambda(create_look_ahead_mask, output_shape=(1, None, None), name='look_ahead_mask')(dec_inputs)
    dec_padding_mask = tf.keras.layers.Lambda(create_padding_mask, output_shape=(1, 1, None), name='dec_padding_mask')(inputs)

    
    enc_outputs = encoder(
        vocab_size=vocab_size,
        num_layers=num_layers,
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
    )
    
    enc_outputs.summary()

    enc_outputs = enc_outputs(inputs=[inputs, enc_padding_mask])
    
    
    dec_outputs = decoder(
        vocab_size=vocab_size,
        num_layers=num_layers,
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
    )
    dec_outputs.summary()
    dec_outputs = dec_outputs(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])
    
    
    outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

    return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

In [134]:
tf.keras.backend.clear_session()

# 하이퍼파라미터
NUM_LAYERS = 2 # 인코더와 디코더의 층의 개수
D_MODEL = 256 # 인코더와 디코더 내부의 입, 출력의 고정 차원
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수 
UNITS = 512 # 피드 포워드 신경망의 은닉층의 크기
DROPOUT = 0.1 # 드롭아웃의 비율

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

model.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 256)    2092800     inputs[0][0]                     
__________________________________________________________________________________________________
tf_op_layer_Mul (TensorFlowOpLa [(None, None, 256)]  0           embedding[0][0]                  
__________________________________________________________________________________________________
positional_encoding (Positional (None, None, 256)    0           tf_op_layer_Mul[0][0]            
____________________________________________________________________________________________

In [135]:
def loss_function(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')(y_true, y_pred)

    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    loss = tf.multiply(loss, mask)

    return tf.reduce_mean(loss)

In [136]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps**-1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [137]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
    return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [138]:
EPOCHS = 20
model.fit(dataset, epochs=EPOCHS, verbose=1)

Epoch 1/20
185/185 [==============================] - 36s 197ms/step - loss: 2.9665 - accuracy: 0.0598
Epoch 2/20
185/185 [==============================] - 36s 197ms/step - loss: 2.4140 - accuracy: 0.1014
Epoch 3/20
185/185 [==============================] - 36s 197ms/step - loss: 2.0499 - accuracy: 0.1037
Epoch 4/20
185/185 [==============================] - 37s 198ms/step - loss: 1.8926 - accuracy: 0.1112
Epoch 5/20
185/185 [==============================] - 37s 197ms/step - loss: 1.7757 - accuracy: 0.1181
Epoch 6/20
185/185 [==============================] - 37s 197ms/step - loss: 1.6511 - accuracy: 0.1265
Epoch 7/20
185/185 [==============================] - 39s 210ms/step - loss: 1.5148 - accuracy: 0.1392
Epoch 8/20
185/185 [==============================] - 37s 203ms/step - loss: 1.3661 - accuracy: 0.1555
Epoch 9/20
185/185 [==============================] - 37s 199ms/step - loss: 1.2079 - accuracy: 0.1733
Epoch 10/20
185/185 [==============================] - 37s 199ms/step - l

In [139]:
def decoder_inference(sentence, tokenizer):
    start_token = [tokenizer.vocab_size]
    end_token = [tokenizer.vocab_size+1]
    sentence = preprocess_sentence(sentence)

    sentence = tf.expand_dims(start_token + tokenizer.encode(sentence) + end_token, axis=0)

    output_sequence = tf.expand_dims(start_token, 0)

    for i in range(MAX_LENGTH):
        predictions = model(inputs=[sentence, output_sequence], training=False)
        predictions = predictions[:, -1:, :]

        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        if tf.equal(predicted_id, end_token[0]):
            break

        output_sequence = tf.concat([output_sequence, predicted_id], axis=-1)

    return tf.squeeze(output_sequence, axis=0)

def sentence_generation(sentence, tokenizer):
  # 입력 문장에 대해서 디코더를 동작 시켜 예측된 정수 시퀀스를 리턴받습니다.
    prediction = decoder_inference(sentence, tokenizer)

  # 정수 시퀀스를 다시 텍스트 시퀀스로 변환합니다.
    predicted_sentence = tokenizer.decode([i for i in prediction if i < tokenizer.vocab_size])

    print('입력 : {}'.format(sentence))
    print('출력 : {}'.format(predicted_sentence))

    return predicted_sentence

In [140]:
sentence_generation('아 몰라', tokenizer)

입력 : 아 몰라
출력 : 반복되은 티를 내봐야죠 .


'반복되은 티를 내봐야죠 .'